## FractionArray numpy interface

In [46]:
from fractions import Fraction

import numpy as np
import attr
from scipy.sparse.sputils import isintlike

DEFAULT_PRECISION = 1e-6

@attr.s
class FractionArray:
    """ Array of rational numbers. """

    numerator: np.ndarray = attr.ib()
    denominator: np.ndarray = attr.ib()
    
    @classmethod
    def from_array(cls, x, precision=DEFAULT_PRECISION):
        raise NotImplemented
        
    def __attrs_post_init__(self):
        if self.numerator.shape != self.denominator.shape:
            raise ValueError("Different numerator and denominator shapes {} vs {}".format(
                self.numerator.shape, self.denominator.shape))
            
        self.shape = self.numerator.shape
        
        self.normalize()
        
    def normalize(self):
        gcds = np.gcd(self.numerator, self.denominator)
          
        self.numerator = self.numerator // gcds
        self.denominator = self.denominator // gcds
        
    def reshape(self, *args, **kwargs):
        return FractionArray(self.numerator.reshape(*args, **kwargs),
                             self.denominator.reshape(*args, **kwargs))
    
    def __abs__(self):
        return FractionArray(abs(self.numerator), self.denominator.copy())
    
    def _add_fraction(self, frac):
        raise NotImplemented("_add_fraction")
        
    def _add_fractionarray(self, frac):
        raise NotImplemented("_add_fraction")
    
    def _add_array(self, frac):
        raise NotImplemented("_add_fraction")
    
    def __add__(self, other):  # self + other
        if isinstance(other, Fraction):
            if other == 0:
                return self.copy()
            
            return self._add_fraction(other)
        
        if isintlike(other):
            if other == 0:
                return self.copy()
            
            return self._add_fraction(Fraction(int(other), 1))
        
        if other.shape != self.shape:
            raise ValueError("inconsistent shapes")
            
        if isinstance(other, FractionArray):
            return self._add_fractionarray(other)
        
        if isinstance(other, np.ndarray):
            if not np.issubdtype(other.dtype, np.integer):
                raise ValueError("Non-integral dtype:", other.dtype)
                
            return self._add_array(other)
        
        raise ValueError("Invalid input to add:", type(other))
                         
    def __radd__(self,other):  # other + self
        return self.__add__(other)
    
    def __len__(self):
        return len(self.numerator)

    def __getitem__(self, key):
        return Fraction(self.numerator[key], self.denominator[key])

    def __setitem__(self, key, value):
        if isinstance(value, Fraction):
            value = value.numerator, value.denominator

        num, denom = value
        self.numerator[key] = num
        self.denominator[key] = denom

    def __iter__(self):
        return iter(
            map(
                lambda p: Fraction(*p), zip(self.numerator, self.denominator)
            )
        )

In [22]:
attr.fields(FractionArray)

(Attribute(name='numerator', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class 'numpy.ndarray'>, converter=None, kw_only=False, inherited=False, on_setattr=None),
 Attribute(name='denominator', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class 'numpy.ndarray'>, converter=None, kw_only=False, inherited=False, on_setattr=None))

In [27]:
fa = FractionArray(np.arange(10), np.ones(10, dtype='int64'))
fb = FractionArray(np.arange(10) - 5, np.ones(10, dtype='int64'))
fa, fb

(FractionArray(numerator=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), denominator=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])),
 FractionArray(numerator=array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4]), denominator=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])))

In [ ]:
fa.reshape((2,5))

In [ ]:
fa

In [14]:
np.arange(10).dtype

dtype('int64')

In [28]:
abs(fb)

FractionArray(numerator=array([5, 4, 3, 2, 1, 0, 1, 2, 3, 4]), denominator=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [43]:
isintlike()

False